In [1]:
import pandas as pd
import re

# Открываем данные

In [3]:
path = 'база данных.csv'

df = pd.read_csv(path)
df = df.fillna(value='')


# Собираем всю найденную информацию в одном файле

In [ ]:
data = pd.read_csv('bibliogorod.csv')

In [ ]:
def clean_name(name):
    name = re.sub('\(.*?\)', '', name)
    name = re.sub('\.', '. ', name)
    name = re.sub('\s\s+', ' ', name)
    name = re.sub('\s$', '', name)
    return name

def find_number(name):
    num = re.findall('№[0-9]+', name)
    if num != []: 
        num = num[0]
        return num
    return False


names, numbers = [], []


for name in data['Название']:
    
    name = clean_name(name)
    names.append(name)
    
    num = find_number(name)
    numbers.append(num)


data['Number'] = numbers
data['Cleaned name'] = names
        

In [ ]:
def find_ind(data, col, name):
    
    res = data.loc[data[col] == name]
    if len(res) != 0:
        ind = res.index[0]
        typ = data['Тип библиотеки'][ind]
        cat = data['Электронный каталог'][ind]
        return typ, cat
    return '', ''


types, catalogs = [], []


for index, name in enumerate(df['Название']):
    
    first = df['Местоположение'][index] == 'Москва'
    second = df['Адрес'][index].startswith('обл Московская')

    if first or second:
        
        name = clean_name(name)
        typ, cat = find_ind(data, 'Cleaned name', name)
        
        if (typ, cat) == ('', ''):
            num = find_number(name)
            typ, cat = find_ind(data, 'Number', num)
    
    else:
        typ, cat = '', ''
        
    types.append(typ)
    catalogs.append(cat)


print(len(types), len(catalogs))
    
df['Тип библиотеки'] = types
df['Электронный каталог'] = catalogs


In [ ]:
paths = {'Изба-читальня': 'izba.txt',
       'Дата открытия': 'open_dates.txt',
        ('Размер фонда', 'Уверенность в фонде'): 'fond_numbers.txt'}


for name in paths:
    
    path = paths[name]
    
    with open(path, 'r', encoding='utf-8') as f:
        
        lines = f.readlines()
        lines = [re.sub('\n', '', i) for i in lines]
        
        if isinstance(name, str):
            df[name] = lines
        else:
            a, b = [], []
            for i in lines:
                if ';' in i:
                    x = i.split(';')
                    fond, conf = x[0], x[1]
                else:
                    fond, conf = '', ''
                    
                a.append(fond)
                b.append(conf)

            df[name[0]] = a
            df[name[1]] = b

In [ ]:
new_data = df[['Тип библиотеки', 'Электронный каталог', 'Изба-читальня',
               'Дата открытия', 'Размер фонда', 'Уверенность в фонде']]

new_data.to_csv('new_info.tsv', sep='\t')